In [ ]:
import torch
import torch.nn as nn

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [ ]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), acc))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Save the model
torch.save(model, 'resnet_trained_model.ckpt')  # 전체 모델 저장
torch.save(model.state_dict(), 'resnet_trained_model_state.ckpt')
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'resnet_trained_model_all.tar')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
from PIL import Image
import torch


# Segmentation을 위한 모델 설정
deeplab101 = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

#transform
transform = transforms.Compose(
    [transforms.ToTensor()])

forwardtransform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

backtransform = transforms.Compose([transforms.Resize((32,32))]) 


#DataLoad
batch_size = 100

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

stack_seg = []
for epoch in range(1):  
    
    for i, data in enumerate(testloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                   
        inputs = forwardtransform(inputs)
        seg = deeplab101(inputs)['out']                
        seg = backtransform(seg) 
        seg = torch.argmax(seg,dim=1)           # Batchsize x W x H 
        #여기까지 하시면 각 Batchsize x W x H 의 각 픽셀에 예측되는 픽셀의 클래스 넘버가 부여
        # 0이 배경이고 나머지 1~ 는 물체
        print(seg.shape)
        stack_seg.append(seg)
        
output = torch.stack(stack_seg, dim=0)
torch.save(output, 'segmented_data.pt')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# data load
batch_size = 100
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, shuffle=False)

original_images = []
for images,_ in test_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_data.pt')
augmented_data = original_data


# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            # image 크기 32x32 중 (m,n)번째 pixel
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    
                    # segmentation label이 0으로 배경인 pixel에 대해
                    # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                    if segmented_data[i][j][m][n] == 0:
                        # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                        #이곳을 바꿔가며 RGBB를 생성
                        augmented_data[i][j][0][m][n] = 0
                        augmented_data[i][j][1][m][n] = 0
                        #augmented_data[i][j][2][m][n] = 0
        print(i+1," batch complete")
        
torch.save(augmented_data, 'blue_augmented_data.pt')

original_images = []
for images,_ in test_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_data.pt')
augmented_data = original_data


# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            # image 크기 32x32 중 (m,n)번째 pixel
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    
                    # segmentation label이 0으로 배경인 pixel에 대해
                    # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                    if segmented_data[i][j][m][n] == 0:
                        # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                        #이곳을 바꿔가며 RGBB를 생성
                        augmented_data[i][j][0][m][n] = 0
                        #augmented_data[i][j][1][m][n] = 0
                        augmented_data[i][j][2][m][n] = 0
        print(i+1," batch complete")
        
torch.save(augmented_data, 'green_augmented_data.pt')

original_images = []
for images,_ in test_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_data.pt')
augmented_data = original_data


# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            # image 크기 32x32 중 (m,n)번째 pixel
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    
                    # segmentation label이 0으로 배경인 pixel에 대해
                    # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                    if segmented_data[i][j][m][n] == 0:
                        # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                        #이곳을 바꿔가며 RGBB를 생성
                        #augmented_data[i][j][0][m][n] = 0
                        augmented_data[i][j][1][m][n] = 0
                        augmented_data[i][j][2][m][n] = 0
        print(i+1," batch complete")
        
torch.save(augmented_data, 'red_augmented_data.pt')

original_images = []
for images,_ in test_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_data.pt')
augmented_data = original_data


# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            # image 크기 32x32 중 (m,n)번째 pixel
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    
                    # segmentation label이 0으로 배경인 pixel에 대해
                    # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                    if segmented_data[i][j][m][n] == 0:
                        # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                        #이곳을 바꿔가며 RGBB를 생성
                        augmented_data[i][j][0][m][n] = 0
                        augmented_data[i][j][1][m][n] = 0
                        augmented_data[i][j][2][m][n] = 0
        print(i+1," batch complete")
        
torch.save(augmented_data, 'black_augmented_data.pt')

In [ ]:
sorted_images_index_150 = []

with torch.no_grad():
    for i in range(100):                               # 100 = len(original_data)
        images_batch = segmented_data[i]
        for j in range(100):                           # 100 = len(images_batch)
            segmented_image = images_batch[j]
            
            """수정된부분"""
            object_pixel=0
            
            for m in range(segmented_image.shape[0]):
                for n in range(segmented_image.shape[1]):
                    if segmented_image[m][n] != 0:
                        object_pixel += 1
                        
            if object_pixel >= 150:
                sorted_images_index_150.append((i*batch_size+(j+1)))
    
print("Thr=150 Done")

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for data in test_loader:                             #100 = len(augmented_data) = 10000 / batch_size
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)    
            
            """수정된부분"""
            #index에 들어있는 사진들만 interfere를 진행
            for j in range(batch_size):
                if( (i*batch_size)+(j+1) in sorted_images_index_150):     
                    total += 1
                    if( predicted[j]==labels[j]):
                        correct += 1

    print('Accuracy of the model on the test images about Thr=150: {} %'.format(100 * correct / total))

In [ ]:
# Thr 적용해서 Testing

#2. Loading Segmented dataset (our dataset)
"""---------------red---------------"""
augmented_data = torch.load('red_augmented_data.pt')
label_set = []
for _,labels in test_loader:
    label_set.append(labels)
augmented_labels = torch.stack(label_set, dim=0)

print(augmented_data.shape)
print(augmented_labels.shape)

# Test the model
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(augmented_data)):
            images = augmented_data[i].to(device)
            labels = augmented_labels[i].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)    
            
            """수정된부분"""
            #index에 들어있는 사진들만 interfere를 진행
            for j in range(batch_size):
                if( (i*batch_size)+(j+1) in sorted_images_index_150):     
                    total += 1
                    if( predicted[j]==labels[j]):
                        correct += 1

    print('Accuracy of the model on the test images about Thr = 150 & red: {} %'.format(100 * correct / total))

In [ ]:
# Thr 적용해서 Testing

#2. Loading Segmented dataset (our dataset)
"""---------------green---------------"""
augmented_data = torch.load('green_augmented_data.pt')
label_set = []
for _,labels in test_loader:
    label_set.append(labels)
augmented_labels = torch.stack(label_set, dim=0)

print(augmented_data.shape)
print(augmented_labels.shape)

# Test the model
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(augmented_data)):
            images = augmented_data[i].to(device)
            labels = augmented_labels[i].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)    
            
            """수정된부분"""
            #index에 들어있는 사진들만 interfere를 진행
            for j in range(batch_size):
                if( (i*batch_size)+(j+1) in sorted_images_index_150):     
                    total += 1
                    if( predicted[j]==labels[j]):
                        correct += 1

    print('Accuracy of the model on the test images about Thr = 150 & green: {} %'.format(100 * correct / total))

In [ ]:
# Thr 적용해서 Testing

#2. Loading Segmented dataset (our dataset)
"""---------------blue---------------"""
augmented_data = torch.load('blue_augmented_data.pt')
label_set = []
for _,labels in test_loader:
    label_set.append(labels)
augmented_labels = torch.stack(label_set, dim=0)

print(augmented_data.shape)
print(augmented_labels.shape)

# Test the model
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(augmented_data)):
            images = augmented_data[i].to(device)
            labels = augmented_labels[i].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)    
            
            """수정된부분"""
            #index에 들어있는 사진들만 interfere를 진행
            for j in range(batch_size):
                if( (i*batch_size)+(j+1) in sorted_images_index_150):     
                    total += 1
                    if( predicted[j]==labels[j]):
                        correct += 1

    print('Accuracy of the model on the test images about Thr = 150 & blue: {} %'.format(100 * correct / total))

In [ ]:
# Thr 적용해서 Testing

#2. Loading Segmented dataset (our dataset)
"""---------------red---------------"""
augmented_data = torch.load('black_augmented_data.pt')
label_set = []
for _,labels in test_loader:
    label_set.append(labels)
augmented_labels = torch.stack(label_set, dim=0)

print(augmented_data.shape)
print(augmented_labels.shape)

# Test the model
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(augmented_data)):
            images = augmented_data[i].to(device)
            labels = augmented_labels[i].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)    
            
            """수정된부분"""
            #index에 들어있는 사진들만 interfere를 진행
            for j in range(batch_size):
                if( (i*batch_size)+(j+1) in sorted_images_index_150):     
                    total += 1
                    if( predicted[j]==labels[j]):
                        correct += 1

    print('Accuracy of the model on the test images about Thr = 150 & black: {} %'.format(100 * correct / total))
    
#여기까지가 Hypothesis Verification

In [ ]:
#### Train dataset Segmentation ####
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
from PIL import Image
import torch


# Segmentation을 위한 모델 설정
deeplab101 = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

#transform
transform = transforms.Compose(
    [transforms.ToTensor()])

forwardtransform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

backtransform = transforms.Compose([transforms.Resize((32,32))]) 


#DataLoad
batch_size = 100

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

stack_seg = []
for epoch in range(1):  
    
    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                   
        inputs = forwardtransform(inputs)
        seg = deeplab101(inputs)['out']                
        seg = backtransform(seg) 
        seg = torch.argmax(seg,dim=1)           # Batchsize x W x H 
        #여기까지 하시면 각 Batchsize x W x H 의 각 픽셀에 예측되는 픽셀의 클래스 넘버가 부여
        # 0이 배경이고 나머지 1~ 는 물체
        print(seg.shape)
        stack_seg.append(seg)
        
print("Segmentation of Trainset Done")
output = torch.stack(stack_seg, dim=0)
print(output.shape)
torch.save(output, 'segmented_train_data.pt')

In [ ]:
#### Augmentation #####
# -- blue -- #
import torch
import torchvision
import torchvision.transforms as transforms

# data load
batch_size = 100
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=True,
                                            transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size, shuffle=False)

original_images = []
for images,_ in train_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_train_data.pt')
augmented_data = original_data
sorted_train_images_index_150 = []

# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            
            object_pixel=0
            
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    if segmented_data[i][j][m][n] != 0:
                        object_pixel += 1
                        
            if object_pixel >= 150:
                            
                # image 크기 32x32 중 (m,n)번째 pixel
                for m in range(image.shape[1]):
                    for n in range(image.shape[2]):
                    
                        # segmentation label이 0으로 배경인 pixel에 대해
                        # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                        if segmented_data[i][j][m][n] == 0:
                            # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                            augmented_data[i][j][0][m][n] = 0
                            augmented_data[i][j][1][m][n] = 0
                            #augmented_data[i][j][2][m][n] = 0
                sorted_train_images_index_150.append((i*batch_size+(j+1)))
        print(i+1," batch complete")
        
print(augmented_data.shape)
torch.save(augmented_data, 'blue_augmented_train_data.pt')
print(len(sorted_train_images_index_150))
torch.save(sorted_train_images_index_150,'sorted_train_images_index_150.pt')       
        
        

# -- Green -- #
original_images = []
for images,_ in train_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_train_data.pt')
augmented_data = original_data
sorted_train_images_index_150 = []

# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            
            object_pixel=0
            
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    if segmented_data[i][j][m][n] != 0:
                        object_pixel += 1
                        
            if object_pixel >= 150:
                            
                # image 크기 32x32 중 (m,n)번째 pixel
                for m in range(image.shape[1]):
                    for n in range(image.shape[2]):
                    
                        # segmentation label이 0으로 배경인 pixel에 대해
                        # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                        if segmented_data[i][j][m][n] == 0:
                            # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                            augmented_data[i][j][0][m][n] = 0
                            #augmented_data[i][j][1][m][n] = 0
                            augmented_data[i][j][2][m][n] = 0
                sorted_train_images_index_150.append((i*batch_size+(j+1)))
        print(i+1," batch complete")
        
print(augmented_data.shape)
torch.save(augmented_data, 'green_augmented_train_data.pt')
print(len(sorted_train_images_index_150))
torch.save(sorted_train_images_index_150,'sorted_train_images_index_150.pt')  




# -- Red -- #
original_images = []
for images,_ in train_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_train_data.pt')
augmented_data = original_data
sorted_train_images_index_150 = []

# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            
            object_pixel=0
            
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    if segmented_data[i][j][m][n] != 0:
                        object_pixel += 1
                        
            if object_pixel >= 150:
                            
                # image 크기 32x32 중 (m,n)번째 pixel
                for m in range(image.shape[1]):
                    for n in range(image.shape[2]):
                    
                        # segmentation label이 0으로 배경인 pixel에 대해
                        # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                        if segmented_data[i][j][m][n] == 0:
                            # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                            #augmented_data[i][j][0][m][n] = 0
                            augmented_data[i][j][1][m][n] = 0
                            augmented_data[i][j][2][m][n] = 0
                sorted_train_images_index_150.append((i*batch_size+(j+1)))
        print(i+1," batch complete")
        
print(augmented_data.shape)
torch.save(augmented_data, 'red_augmented_train_data.pt')
print(len(sorted_train_images_index_150))
torch.save(sorted_train_images_index_150,'sorted_train_images_index_150.pt')  



# -- Black -- #
original_images = []
for images,_ in train_loader:
    original_images.append(images)

original_data = torch.stack(original_images, dim=0)
segmented_data = torch.load('segmented_train_data.pt')
augmented_data = original_data
sorted_train_images_index_150 = []

# data augmentation
with torch.no_grad():
    for i in range(len(original_data)):
        images_batch = original_data[i]
        for j in range(len(images_batch)):
            image = images_batch[j]
            
            
            object_pixel=0
            
            for m in range(image.shape[1]):
                for n in range(image.shape[2]):
                    if segmented_data[i][j][m][n] != 0:
                        object_pixel += 1
                        
            if object_pixel >= 150:
                            
                # image 크기 32x32 중 (m,n)번째 pixel
                for m in range(image.shape[1]):
                    for n in range(image.shape[2]):
                    
                        # segmentation label이 0으로 배경인 pixel에 대해
                        # RGB 모두 0으로 설정해 검정 배경으로 만들어줌
                        if segmented_data[i][j][m][n] == 0:
                            # i번째 batch의 j번째 image의 (m,n) 위치의 RGB intensity값 0으로 설정
                            augmented_data[i][j][0][m][n] = 0
                            augmented_data[i][j][1][m][n] = 0
                            augmented_data[i][j][2][m][n] = 0
                sorted_train_images_index_150.append((i*batch_size+(j+1)))
        print(i+1," batch complete")
        
print(augmented_data.shape)
torch.save(augmented_data, 'black_augmented_train_data.pt')
print(len(sorted_train_images_index_150))
torch.save(sorted_train_images_index_150,'sorted_train_images_index_150.pt')  

In [ ]:
# Prepare Fine-Tuning dataset

# Extract labels of train data corresponding to Thr = 150
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 1. Prepare labels of augmented dats

augmented_images_label = []
augmented_index = torch.load('sorted_train_images_index_150.pt')
print(len(augmented_index))


batch_size = 100

#CIFAR-10
transform = transforms.Compose([transforms.ToTensor()])

train_data = torchvision.datasets.CIFAR10(root='../../data/',
                                         train=True,
                                         transform = transform,
                                         download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                          shuffle=False,
                                          batch_size = batch_size)

for epoch in range(1):
    
    for i,data in enumerate(train_loader):
        
        images,labels = data
        for j in range(batch_size):
            if ( i*batch_size + (j+1) )in augmented_index:
                augmented_images_label.append(labels[j])

                
print( "augmented_images_labels has ",len(augmented_images_label) ," labels")

torch.save(augmented_images_label,'augmented_images_label.pt')

In [ ]:
human_labor_index_0_500 = [5,8,9,10,12,13,15,19,27,29,30,33,35,38,40,41,43,44,45,47,50,52,53,56,62,66,69,71,78,82,84,85,87,88,92,100]
human_labor_index_1_500 = [1,3,6,15,16,20,21,24,27,29,34,37,38,39,46,47,53,56,57,59,61,64,73,76,77,78,81,83,85,95]
human_labor_index_2_500 = [4,7,12,13,14,16,23,27,32,34,37,38,39,51,56,57,58,61,63,67,76,79,80,81,82,90,91,95,98]
human_labor_index_3_500 = [5,6,12,13,16,19,21,24,25,26,30,31,32,38,40,42,44,47,55,58,63,73,75,76,81,83,84,89,92,95]
human_labor_index_4_500 = [6,11,13,14,17,23,27,33,41,50,62,64,69,70,80,83,87,92,93]
human_labor_index_5_500 = [3,4,13,23,32,36,48,51,52,61,62,69,71,77,79,96,99]
human_labor_index_6_500 = [6,7,37,42,44,45,56,63,66,69,76,77,87,92,97,99]
human_labor_index_7_500 = [27,30,31,33,36,40,43,44,45,47,51,61,63,66,72,73,76,77,80,84,85,90,91,95,98,100]
human_labor_index_8_500 = [3,7,9,10,14,18,23,24,26,28,31,41,43,46,47,49,53,56,57,72,82,86,90,98]
human_labor_index_9_500 = [9,14,17,18,20,23,30,37,41,43,49,54,57,63,69,73,78,79,81,92,99]
human_labor_index_10_500 = [18,20,21,22,23,25,43,47,57,62,65,67,75,80,81,82,84,85,91,94,95]
human_labor_index_11_500 = [18,22,30,34,35,46,54,57,66,68,73,74,80,82,90,97]
human_labor_index_12_500 = [16,20,27,30,41,43,52,58,59,72,76,81,88]
human_labor_index_13_500 = [21,29,37,38,48,57,59,70,73,81]
human_labor_index_14_500 = [11,12,14,15,18,20,27,46,49,51,52,56,58,59,60,61,63,68,77,83,84,88,90,91,95,97,99]
human_labor_index_15_500 = [3,13,14,19,26,28,29,31,44,46,47,52,57,60,62,63,66,67,75,83,84,89,95,96]
human_labor_index_16_500 = [9,15,16,18,22,35,48,51,54,57,63,64,80,82,92,95,100]
human_labor_index_17_500 = [1,14,18,24,28,29,37,39,41,43,58,59,77,78,79,81,82,86,93]
human_labor_index_18_500 = [7,9,16,17,18,21,36,41,42,49,52,53,54,59,66,70,73,91,92,95,98,99,100]
human_labor_index_19_500 = [1,2,5,19,28,31,32,47,50,51,52,62,74,81,98]
human_labor_index_20_500 = [1,9,10,12,16,20,31,38,40,58,60,64,65,84,89,91,95]
human_labor_index_21_500 = [5,9,10,36,37,39,40,41,46,49,50,54,64,78,81,86,96,99]
human_labor_index_22_500 = [1,6,8,13,22,28,35,37,42,43,44,45,50,57,58,61,69,71,79,81]
human_labor_index_23_500 = [21,22,29,32,36,37,44,49,50,59,63,65,68,79,83,95,100]
human_labor_index_24_500 = [4,6,11,23,28,29,31,39,41,47,54,56,57,58,62,64,67,70,72,74,75,81,90,92,94]
human_labor_index_25_500 = [1,12,13,18,19,23,42,46,47,55,63,70,75,78,83,84,85,86,88,89,93,94,98]
human_labor_index_26_500 = [1,2,9,12,13,18,21,26,29,30,31,35,40,41,52,53,54,57,61,62,63,78,83,87,90,97]
human_labor_index_27_500 = [1,2,12,28,29,31,36,37,39,45,46,49,51,63,64,74,81,82,84,89,92,95,96,100]
human_labor_index_28_500 = [16,17,18,20,23,26,27,28,35,44,45,53,55,61,63,65,71,79,88,90,95]
human_labor_index_29_500 = [13,16,30,31,39,43,47,49,55,64,70,76,78,82,87,89,94 ]
human_labor_index_30_500 = [2,5,6,16,21,22,24,27,30,35,38,41,52,55,61,67,72,73,74,86,88,90]
human_labor_index_31_500 = [12,13,14,15,19,36,39,42,47,49,54,58,59,63,65,75,82,84,85,89,91,94,95,97,99]
human_labor_index_32_500 = [9,10,11,23,32,39,62,73,81,85,86,90,92,98]
human_labor_index_33_500 = [10,18,27,29,30,31,46,50,53,57,59,61,68,69,71,86,87,88,91,99,100]
human_labor_index_34_500 = [1,5,11,14,18,20,24,26,27,29,33,35,38,42,53,67,75,77,84,85,86,89,93,99]
human_labor_index_35_500 =[1,13,32,44,49,52,53,56,65,69,73,74,81,82,84,97]
human_labor_index_36_500 =[2,6,10,11,12,15,18,23,33,37,38,44,51,52,55,60,63,74,81,82,88,91,93,95,97,100]
human_labor_index_37_500 =[2,5,9,10,12,14,29,36,41,44,45,50,57,63,67,69,74,80,82,87,88,97,100]
human_labor_index_38_500 = [7,8,15,18,19,25,28,33,36,37,87,88,93,95,99,100]
human_labor_index_39_500 = [2,6,8,10,23,26,34,35,38,40,55,56,61,64,76,78,89,96]
human_labor_index_40_500 = [1,2,12,14,21,36,38,41,55,60,64,71,74,76,77,79,85,86,99]
human_labor_index_41_500 = [1,15,17,34,35,36,41,55,56,59,62,65,72,78,88,93]
human_labor_index_42_500 = [1,13,17,21,29,37,38,45,50,51,52,56,62,82,86,89]
human_labor_index_43_500 = [20,26,32,34,42,49,56,58,62,75,76,78,79,80,86,88,90,92]
human_labor_index_44_500 = [2,19,20,24,33,34,40,45,49,52,69,86,87,93,95]
human_labor_index_45_500 = [1,6,9,11,13,14,18,19,24,26,30,35,41,54,60,65,83,84,93,98,99]
human_labor_index_46_500 = [25,32,37,48,51,70,80,87,88,90,98,100]
human_labor_index_47_500 = [13,15,21,22,33,37,40,47,48,49,59,62,65,76,83,90,91,96,98]
human_labor_index_48_500 = [12,29,31,34,36,45,59,74,75,82,84,92,93,95]
human_labor_index_49_500 = [2,18,29,35,40,42,43,45,46,50,54,59,69,78,82,84,88,96,98]

index50 = [human_labor_index_0_500 ,
human_labor_index_1_500 ,
human_labor_index_2_500 ,
human_labor_index_3_500 ,
human_labor_index_4_500,
human_labor_index_5_500 ,
human_labor_index_6_500,
human_labor_index_7_500 ,
human_labor_index_8_500 ,
human_labor_index_9_500,
human_labor_index_10_500,
human_labor_index_11_500,
human_labor_index_12_500,
human_labor_index_13_500 ,
human_labor_index_14_500,
human_labor_index_15_500 ,
human_labor_index_16_500 ,
human_labor_index_17_500 ,
human_labor_index_18_500,
human_labor_index_19_500,
human_labor_index_20_500 ,
human_labor_index_21_500 ,
human_labor_index_22_500 ,
human_labor_index_23_500,
human_labor_index_24_500 ,
human_labor_index_25_500,
human_labor_index_26_500 ,
human_labor_index_27_500 ,
human_labor_index_28_500 ,
human_labor_index_29_500 ,
human_labor_index_30_500 ,
human_labor_index_31_500 ,
human_labor_index_32_500 ,
human_labor_index_33_500 ,
human_labor_index_34_500,
human_labor_index_35_500 ,
human_labor_index_36_500 ,
human_labor_index_37_500 ,
human_labor_index_38_500 ,
human_labor_index_39_500 ,
human_labor_index_40_500 ,
human_labor_index_41_500 ,
human_labor_index_42_500 ,
human_labor_index_43_500 ,
human_labor_index_44_500 ,
human_labor_index_45_500 ,
human_labor_index_46_500 ,
human_labor_index_47_500,
human_labor_index_48_500 ,
human_labor_index_49_500 ]

print(len(index50))


A = []
total=0

for i in range(len(index50)):
    indices = index50[i]
    total += len(indices)
    for j in range(len(indices)):
        A.append(indices[j]+100*i)
        
print(len(A))
print(total)


torch.save(A,'human_labor_index.pt')
torch.save(index50,'index50.pt')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import random

# CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=False)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epoch = 60
learning_rate = 0.0001
batch_size = 100


#1015개에 해당하는 label들만 뽑아내기
human_labor_label = []

index50 = torch.load('index50.pt')

for epoch in range(1):  
    
    for i, data in enumerate(train_loader, 0):

        if(i<=49):
            inputs, labels = data
            for j in range(len(index50[i])):
                human_labor_label.append( labels[ index50[i][j]-1 ] )
    
print(len(human_labor_label))
torch.save(human_labor_label,'human_labor_label1015.pt')

In [ ]:
# We prepare everything
# Trainings From now on 

# Transfer Learning = 0 / Thr = 150 / RRGB

import torch
import torchvision
import torchvision.transforms as transforms
import random

# Settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100

# Image & Label Setting 4x28519 & 4x28519


blue_augmented_data = torch.load('blue_augmented_train_data.pt')
green_augmented_data = torch.load('green_augmented_train_data.pt')
red_augmented_data = torch.load('red_augmented_train_data.pt')
black_augmented_data = torch.load('black_augmented_train_data.pt')
sorted_train_images_index_150 = torch.load('sorted_train_images_index_150.pt')
augmented_images_label = torch.load('augmented_images_label.pt')
print(blue_augmented_data.shape)




whole_image = []
whole_label = augmented_images_label + augmented_images_label + augmented_images_label + augmented_images_label

# Blue
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(blue_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Blue : ",i+1,"/500 completed")
        

# Green
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(green_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Green : ",i+1,"/500 completed")

        
# Red
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(red_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Red : ",i+1,"/500 completed")

        
# Black
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(black_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Black : ",i+1,"/500 completed")

        
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print( "size of whole_image is",whole_image.size())
print( "size of whole_label is",whole_label.size())







import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 20
learning_rate = 0.001

# Model Load
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, rho=0.9, eps=1e-06, weight_decay=0)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        

        

        

        
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate

print(whole_image.size())
print(whole_label.size())

model.train()

for epoch in range(num_epoch):
    model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))
        
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                

            
            
            
# 1. Loading CIFAR dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)


# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))


In [ ]:
# Transfer Learning = 0 / Thr = Human / RRGB


blue_augmented_data = torch.load('blue_augmented_train_data.pt')
green_augmented_data = torch.load('green_augmented_train_data.pt')
red_augmented_data = torch.load('red_augmented_train_data.pt')
black_augmented_data = torch.load('black_augmented_train_data.pt')
print(blue_augmented_data.shape)

human_labor_index = torch.load('human_labor_index.pt')
index50 = torch.load('index50.pt')
human_labor_label1015 = torch.load('human_labor_label1015.pt')


whole_image = []
whole_label = human_labor_label1015 + human_labor_label1015 + human_labor_label1015 + human_labor_label1015

# Blue
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(blue_augmented_data[i][j])
            
print("Blue completed")
        

# Green
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(green_augmented_data[i][j])
            
print("Green completed")

        
# Red
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(red_augmented_data[i][j])
            
print("Red completed")

        
# Black
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(black_augmented_data[i][j])
            
print("Black completed")

        
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print( "size of whole_image is",whole_image.size())
print( "size of whole_label is",whole_label.size())
torch.save(whole_image,'ft_data_image_1015.pt')






import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock


# Model Load
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
        
        
        
        
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate

print(whole_image.size())
print(whole_label.size())

model.train()

for epoch in range(num_epoch):
    model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 10 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))
        
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

        
        
        

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))


In [ ]:
# Transfer Learning = X / Thr = 150 / RRGB


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100

# CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])


train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100,
                                          shuffle=False)




# Image & Label Setting 4x28519 & 4x28519


blue_augmented_data = torch.load('blue_augmented_train_data.pt')
green_augmented_data = torch.load('green_augmented_train_data.pt')
red_augmented_data = torch.load('red_augmented_train_data.pt')
black_augmented_data = torch.load('black_augmented_train_data.pt')
sorted_train_images_index_150 = torch.load('sorted_train_images_index_150.pt')
augmented_images_label = torch.load('augmented_images_label.pt')
print(blue_augmented_data.shape)


whole_image = []
whole_label = augmented_images_label + augmented_images_label + augmented_images_label + augmented_images_label

# Blue
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(blue_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Blue : ",i+1,"/500 completed")
        

# Green
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(green_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Green : ",i+1,"/500 completed")

        
# Red
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(red_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Red : ",i+1,"/500 completed")

        
# Black
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(black_augmented_data[i][j])
            
    if( (i+1)%10 ==0 ):
        print("Black : ",i+1,"/500 completed")

        
print( "length of whole_image is",len(whole_image))
print( "length of whole_label is",len(whole_label))




#50000 from CIFAR-10 Train Data
with torch.no_grad():
    for i,data in enumerate(train_loader):
        images,labels = data
        for j in range(images.size()[0]):
            whole_image.append(images[j])
            whole_label.append(labels[j])
print("length of whole_image = ",len(whole_image))
print("length of whole_label = ",len(whole_label))


# Model
from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 80
learning_rate = 0.001

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adadelta(model.parameters(), lr=1, rho=0.9, eps=1e-06, weight_decay=0)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
        
# Training
total_step = len(whole_image) // batch_size
curr_lr = learning_rate
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print(whole_image.size())
print(whole_label.size())

model.train()

for epoch in range(num_epoch):
    #model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))
            
            
# Test
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)

model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the pre-trained model on the test images: {} %'.format(100 * correct / total))

In [ ]:
# Transfer Learning = X / Thr = Human / RRGB


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100

# CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=False)

index50 = torch.load('index50.pt')
human_labor_label1015 = torch.load('human_labor_label1015.pt')
human_labor_index = torch.load('human_labor_index.pt')

print('len(index50) : ',len(index50))
print('len(human_labor_index) : ',len(human_labor_index))
print('len(human_labor_label1015) : ',len(human_labor_label1015))



# Prepare Augmented Data

blue_augmented_data = torch.load('blue_augmented_train_data.pt')
green_augmented_data = torch.load('green_augmented_train_data.pt')
red_augmented_data = torch.load('red_augmented_train_data.pt')
black_augmented_data = torch.load('black_augmented_train_data.pt')
print(blue_augmented_data.shape)


whole_image = []
whole_label = human_labor_label1015 + human_labor_label1015 + human_labor_label1015 + human_labor_label1015

# Blue
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(blue_augmented_data[i][j])
            
print("Blue completed")
        

# Green
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(green_augmented_data[i][j])
            
print("Green completed")

        
# Red
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(red_augmented_data[i][j])
            
print("Red completed")

        
# Black
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(black_augmented_data[i][j])
            
print("Black completed")

        
print( "length of whole_image is",len(whole_image))
print( "length of whole_label is",len(whole_label))

# Extract 50000 from CIAFR-10 Train Data
with torch.no_grad():
    for i,data in enumerate(train_loader):
        images,labels = data
        for j in range(images.size()[0]):
            whole_image.append(images[j])
            whole_label.append(labels[j])
print("length of whole_image = ",len(whole_image))
print("length of whole_label = ",len(whole_label))


# Model
from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 160
learning_rate = 0.001

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adadelta(model.parameters(), lr=1, rho=0.9, eps=1e-06, weight_decay=0)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
        
# Training     
total_step = len(whole_image) // batch_size
curr_lr = learning_rate
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print(whole_image.size())
print(whole_label.size())

model.train()

for epoch in range(num_epoch):
    #model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))
        
# Test
#Test the pre_trained Model

# 1. Loading CIFAR dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)

model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
# Transfer Learning = 0 / Thr = 150 / 7 Background

import torch
import torchvision
import torchvision.transforms as transforms
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os

# Settings & Load

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100


background1 = torch.load('background1.pt')
background2 = torch.load('background2.pt')
background3 = torch.load('background3.pt')
background4 = torch.load('background4.pt')
background5 = torch.load('background5.pt')
background6 = torch.load('background6.pt')
background7 = torch.load('background7.pt')

sorted_train_images_index_150 = torch.load('sorted_train_images_index_150.pt')
augmented_images_label = torch.load('augmented_images_label.pt')


# Image & Label Setting 4x28519 & 4x28519
# Label

whole_label = []
for i in range(7):
    whole_label += augmented_images_label

    
# Image
whole_image = []
  

# backgound1
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background1[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background2
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background2[i][j])
    if ((i+1) % 100 ==0) :
        print("{}/500 Done".format(i+1))
        
# background3
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background3[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background4
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background4[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background5
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background5[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background6
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background6[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background7
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background7[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            

print("whole_image = ", len(whole_image))
print("whole_label = ", len(whole_label))
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print(whole_image.size())
print(whole_label.size())
print("28519*7 = ",28519*7)


# Model 
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 80
learning_rate = 0.001

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, rho=0.9, eps=1e-06, weight_decay=0)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
# Training
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate

print(whole_image.size())
print(whole_label.size())

model.train()

max_acc = 0

for epoch in range(num_epoch):
    model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))
        
        if max_acc < 100 * correct / total :
            max_acc = 100 * correct / total
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                
print(max_acc)

# Test the model
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)



model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))


In [ ]:
# Tranfer Learning = 0 / Thr = Human / 7 Backgrounds

import torch
import torchvision
import torchvision.transforms as transforms
import random

# CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=False)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epoch = 80
learning_rate = 0.001
batch_size = 100


background1 = torch.load('background1.pt')
background2 = torch.load('background2.pt')
background3 = torch.load('background3.pt')
background4 = torch.load('background4.pt')
background5 = torch.load('background5.pt')
background6 = torch.load('background6.pt')
background7 = torch.load('background7.pt')

human_labor_index = torch.load('human_labor_index.pt')
#sorted_train_images_index_150 = torch.load('sorted_train_images_index_150.pt')
human_labor_label1015 = torch.load('human_labor_label1015.pt')




# Image & Label 


# Label

whole_label = []
for i in range(7):
    whole_label += human_labor_label1015

    
    
# Image

whole_image = []
  

# backgound1
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background1[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background2
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background2[i][j])
    if ((i+1) % 100 ==0) :
        print("{}/500 Done".format(i+1))
        
# background3
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background3[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background4
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background4[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background5
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background5[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background6
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background6[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background7
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background7[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            

print("whole_image = ", len(whole_image))
print("whole_label = ", len(whole_label))
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print(whole_image.size())
print(whole_label.size())
print("28519*7 = ",28519*7)






import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock


# Model Load
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model = torch.load('resnet_trained_model.ckpt')
model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
        
        
        
        
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate

print(whole_image.size())
print(whole_label.size())

model.train()

max_acc = 0

for epoch in range(num_epoch):
    model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 10 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))
     
    if max_acc < 100 * correct / total :
        max_acc = 100 * correct / total
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                
            
print(max_acc)

In [ ]:
# Transfer Learning = X / Thr = 150 / 7 Backgrounds

import torch
import torchvision
import torchvision.transforms as transforms
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os

# Settings & Load

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100


background1 = torch.load('background1.pt')
background2 = torch.load('background2.pt')
background3 = torch.load('background3.pt')
background4 = torch.load('background4.pt')
background5 = torch.load('background5.pt')
background6 = torch.load('background6.pt')
background7 = torch.load('background7.pt')

sorted_train_images_index_150 = torch.load('sorted_train_images_index_150.pt')
augmented_images_label = torch.load('augmented_images_label.pt')




# CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100,
                                          shuffle=False)

whole_image = []
whole_label = []

with torch.no_grad():
    for i,data in enumerate(train_loader):
        images,labels = data
        for j in range(images.size()[0]):
            whole_image.append(images[j])
            whole_label.append(labels[j])
print("length of whole_image = ",len(whole_image))
print("length of whole_label = ",len(whole_label))





# Image & Label Setting 4x28519 & 4x28519


# Label

#whole_label = []
for i in range(7):
    whole_label += augmented_images_label

    
    
# Image

#whole_image = []
  

# backgound1
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background1[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background2
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background2[i][j])
    if ((i+1) % 100 ==0) :
        print("{}/500 Done".format(i+1))
        
# background3
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background3[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background4
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background4[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background5
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background5[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background6
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background6[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background7
for i in range(500):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in sorted_train_images_index_150:
            whole_image.append(background7[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            

print("whole_image = ", len(whole_image))
print("whole_label = ", len(whole_label))
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print(whole_image.size())
print(whole_label.size())
print("28519*7 + 50000= ",28519*7 + 50000)







import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 200
learning_rate = 0.001

# Model Load
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
#model = torch.load('resnet_trained_model.ckpt')
#model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, rho=0.9, eps=1e-06, weight_decay=0)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
        
        
        
        
        
        
        
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate

print(whole_image.size())
print(whole_label.size())

model.train()

max_acc = 0

for epoch in range(num_epoch):
    model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))
        
        if max_acc < 100 * correct / total :
            max_acc = 100 * correct / total
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                
print(max_acc)

In [ ]:
# Transfer Learning = X / Thr = Human / 7 Backgrounds

import torch
import torchvision
import torchvision.transforms as transforms
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os

# Settings & Load

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 100


background1 = torch.load('background1.pt')
background2 = torch.load('background2.pt')
background3 = torch.load('background3.pt')
background4 = torch.load('background4.pt')
background5 = torch.load('background5.pt')
background6 = torch.load('background6.pt')
background7 = torch.load('background7.pt')

sorted_train_images_index_150 = torch.load('sorted_train_images_index_150.pt')
augmented_images_label = torch.load('augmented_images_label.pt')

index50 = torch.load('index50.pt')
human_labor_label1015 = torch.load('human_labor_label1015.pt')
human_labor_index = torch.load('human_labor_index.pt')

print('len(index50) : ',len(index50))
print('len(human_labor_index) : ',len(human_labor_index))
print('len(human_labor_label1015) : ',len(human_labor_label1015))




# CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100,
                                          shuffle=False)

whole_image = []
whole_label = []

with torch.no_grad():
    for i,data in enumerate(train_loader):
        images,labels = data
        for j in range(images.size()[0]):
            whole_image.append(images[j])
            whole_label.append(labels[j])
print("length of whole_image = ",len(whole_image))
print("length of whole_label = ",len(whole_label))




# Image & Label Setting 4x28519 & 4x28519


# Label

#whole_label = []
for i in range(7):
    whole_label += human_labor_label1015

    
    
# Image

#whole_image = []
  

# backgound1
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background1[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background2
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background2[i][j])
    if ((i+1) % 100 ==0) :
        print("{}/500 Done".format(i+1))
        
# background3
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background3[i][j])
    if ((i+1) % 100 == 0) :
        print("{}/500 Done".format(i+1))
            
# background4
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background4[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background5
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background5[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background6
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background6[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            
# background7
for i in range(50):
    for j in range(100):    
        if ( i*batch_size + (j+1) ) in human_labor_index:
            whole_image.append(background7[i][j])
    if ((i+1) % 100 == 0 ) :
        print("{}/500 Done".format(i+1))
            

print("whole_image = ", len(whole_image))
print("whole_label = ", len(whole_label))
whole_image = torch.stack(whole_image)
whole_label = torch.stack(whole_label)
print(whole_image.size())
print(whole_label.size())
print("1015*7 +50000= ",1015*7 + 50000)





import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from model_resnet import ResNet
from model_resnet import ResidualBlock

num_epoch = 160
learning_rate = 0.001

# Model Load
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
#model = torch.load('resnet_trained_model.ckpt')
#model.load_state_dict(torch.load('resnet_trained_model_state.ckpt'))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, rho=0.9, eps=1e-06, weight_decay=0)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
        
        
        
        
total_step = whole_image.size()[0] // batch_size
curr_lr = learning_rate

print(whole_image.size())
print(whole_label.size())

model.train()

max_acc = 0

for epoch in range(num_epoch):
    model.train()
    correct = 0
    total = 0
    
    # GPU & Shuffle & Transform
    whole_image = whole_image.to(device)
    whole_label = whole_label.to(device)
    
    
    torch.manual_seed(epoch)
    pmtn = torch.randperm(whole_image.size()[0])          #seed를 고정해도 torch.randperm이 선언될때마다 각각은 고정되지만
    whole_image=whole_image[pmtn]                         #서로 다른 pmtn을 갖는다 그래서 하나로 고정
    whole_label=whole_label[pmtn]    
    
    whole_image0 = transforms.RandomCrop(32)(transforms.RandomHorizontalFlip()(transforms.Pad(4)(whole_image)))

    for i in range(whole_image0.size()[0] // batch_size):
        images = whole_image0[i*batch_size:(i+1)*batch_size]
        labels = whole_label[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += labels.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

        acc = 100 * correct / total
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} | Accuracy: {:.4f}"
                   .format(epoch+1, num_epoch, i+1, total_step, loss.item(), acc))
            
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the fine-tuned model on the test images: {} %'.format(100 * correct / total))
        
        if max_acc < 100 * correct / total :
            max_acc = 100 * correct / total
        
     # Decay learning rate
    if ((epoch+1)%20==0):
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
                

            
print(max_acc)